In [1]:
# !pip install requests
import requests, sys, time, os, argparse
import pandas as pd
import numpy as np
import pymysql

In [2]:
con = pymysql.connect(
    host='host',
    user='user',
    password='passwd',
    db='db_name',
    charset='utf8mb4', 
    autocommit=True, 
    cursorclass=pymysql.cursors.DictCursor 
)
cur = con.cursor()

In [3]:
snippet_features = ["title",
                    "publishedAt",
                    "channelId",
                    "channelTitle",
                    "categoryId"]

unsafe_characters = ['\n', '"']

header = ["video_id"] + snippet_features + ["trending_date", "tags", "view_count", "likes", "dislikes",
                                            "comment_count", "thumbnail_link", "comments_disabled",
                                            "ratings_disabled", "description"]


def setup(api_path, code_path):
    api_key = 'Key'
    country_codes = ['KR']
    
    return api_key, country_codes


def prepare_feature(feature):
    for ch in unsafe_characters:
        feature = str(feature).replace(ch, "")
    return f'"{feature}"'


def api_request(page_token, country_code):
    # Builds the URL and requests the JSON from it
    request_url = f"https://www.googleapis.com/youtube/v3/videos?part=id,statistics,snippet{page_token}chart=mostPopular&regionCode={country_code}&maxResults=50&key={api_key}"
    request = requests.get(request_url)
    if request.status_code == 429:
        print("Temp-Banned due to excess requests, please wait and continue later")
        sys.exit()
    return request.json()


def get_tags(tags_list):
    return prepare_feature("|".join(tags_list))


def get_videos(items):
    lines = []
    for video in items:
        comments_disabled = False
        ratings_disabled = False

        if "statistics" not in video:
            continue

        video_id = prepare_feature(video['id'])

        snippet = video['snippet']
        statistics = video['statistics']

        features = [prepare_feature(snippet.get(feature, "")) for feature in snippet_features]

        description = snippet.get("description", "")
        thumbnail_link = snippet.get("thumbnails", dict()).get("default", dict()).get("url", "")
        trending_date = time.strftime("%y.%d.%m")
        tags = get_tags(snippet.get("tags", ["[none]"]))
        view_count = statistics.get("viewCount", 0)

        if 'likeCount' in statistics and 'dislikeCount' in statistics:
            likes = statistics['likeCount']
            dislikes = statistics['dislikeCount']
        else:
            ratings_disabled = True
            likes = 0
            dislikes = 0

        if 'commentCount' in statistics:
            comment_count = statistics['commentCount']
        else:
            comments_disabled = True
            comment_count = 0

        line = [video_id] + features + [prepare_feature(x) for x in [trending_date, tags, view_count, likes, dislikes,
                                                                       comment_count, thumbnail_link, comments_disabled,
                                                                       ratings_disabled, description]]
        lines.append(",".join(line))
    return lines


def get_pages(country_code, next_page_token="&"):
    country_data = []

    while next_page_token is not None:
        video_data_page = api_request(next_page_token, country_code)

        next_page_token = video_data_page.get("nextPageToken", None)
        next_page_token = f"&pageToken={next_page_token}&" if next_page_token is not None else next_page_token

        items = video_data_page.get('items', [])
        country_data += get_videos(items)

    return country_data



def write_to_file(country_code, country_data):
    
    print(f"Writing {country_code} data to file...")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(f"{output_dir}/{time.strftime('%y.%d.%m')}_{country_code}_videos.csv", "w+", encoding='utf-8') as file:
        for row in country_data:
            file.write(f"{row}\n")
                

def get_data():
    for country_code in country_codes:
        country_data = [",".join(header)] + get_pages(country_code)
        write_to_file(country_code, country_data)
    
    return f"{output_dir}/{time.strftime('%y.%d.%m')}_{country_code}_videos.csv"

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--key_path', help='Path to the file containing the api key, by default will use api_key.txt in the same directory', default='api_key.txt')
parser.add_argument('--country_code_path', help='Path to the file containing the list of country codes to scrape, by default will use country_codes.txt in the same directory', default='country_codes.txt')
parser.add_argument('--output_dir', help='Path to save the outputted files in', default='output/')

# args = parser.parse_args()
args = parser.parse_args(args=[]) #Jupyter

output_dir = args.output_dir
api_key, country_codes = setup(args.key_path, args.country_code_path)

csv_path = get_data()

Writing KR data to file...


In [5]:
df = pd.read_csv(csv_path)
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,RaiE-_O_lXk,[안방1열 풀캠4K] 블랙핑크 'Pink Venom' (BLACKPINK FullC...,2022-08-28T08:17:03Z,UCS_hnpJLQTvBkqALgapi_4g,스브스케이팝 X INKIGAYO,24,22.29.08,Inkigayo|인기가요직캠|안방1열직캠|스브스케이팝|스브스 직캠|인기가요|인가|음...,4347602,0,0,10739,https://i.ytimg.com/vi/RaiE-_O_lXk/default.jpg,False,True,[안방1열 풀캠4K] 블랙핑크 'Pink Venom' (BLACKPINK FullC...
1,49tjroGV8nk,도쿄의 고독한 대식가 - 일본(1),2022-08-28T12:59:35Z,UClRNDVO8093rmRTtLe4GEPw,곽튜브KWAKTUBE,19,22.29.08,일본|일본 여행|도쿄|일본 먹방|먹방,645415,0,0,2713,https://i.ytimg.com/vi/49tjroGV8nk/default.jpg,False,True,안녕하세요'디스이즈네버댓'의 초청으로 일본에 잠시 왔습니다.당분간 일본 먹방여행으로...
2,GOh-qDEN5jU,Resumen de Rayo Vallecano vs RCD Mallorca (0-2),2022-08-27T20:58:01Z,UCTv-XvfzLX3i4IGWAm4sbmA,LaLiga Santander,17,22.29.08,liga|laliga|laliga santander|football|resumen|...,467747,0,0,531,https://i.ytimg.com/vi/GOh-qDEN5jU/default.jpg,False,True,El RCD Mallorca se impone al Rayo Vallecano a ...
3,7bs9-VU8uHI,커피 좀 타 드릴까요?,2022-08-27T06:45:28Z,UCXEKwWflysXu312NmIP_dlw,너덜트,23,22.29.08,미생|회사|직장상사|신입사원|꼰대|복수|커피|탕비실|코믹숏무비,1400045,0,0,1431,https://i.ytimg.com/vi/7bs9-VU8uHI/default.jpg,False,True,#코믹미들무비 #nerdult #소심한복수복수를 꿈꾸는 신입 사원의 고군분투신입사원...
4,STTEIUrP9y8,[안방1열 직캠4K] 블랙핑크 제니 'Pink Venom' (BLACKPINK JE...,2022-08-28T09:00:21Z,UCM3jwNRfl5-W8VzgT6DsaEQ,스브스케이팝 ZOOM,22,22.29.08,블랙핑크|BLACKPINK|Pink Venom|블랙핑크 컴백|블랙핑크 인기가요|블랙...,862893,0,0,5428,https://i.ytimg.com/vi/STTEIUrP9y8/default.jpg,False,True,[안방1열 직캠4K] 블랙핑크 제니 'Pink Venom' (BLACKPINK JE...


In [6]:
df = df.replace({np.nan: None})
df = df.replace({'[none]': None})

In [7]:
df.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

In [8]:
query = """
    INSERT INTO youtube_data2
    (seq, video_id, trending_date, title, channel_title, category_id,
    publish_time, tags, views, likes, dislikes, comment_count,
    thumbnail_link, comments_disabled, ratings_disabled,
    description)
    VALUES (0,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """

for col, item in df.iterrows():
    cur.execute(query,(item['video_id'],item['trending_date'],item['title'],item['channelTitle'],
                       item['categoryId'],item['publishedAt'],item['tags'],item['view_count'],
                       item['likes'],item['dislikes'],item['comment_count'],item['thumbnail_link'],
                       item['comments_disabled'],item['ratings_disabled'],item['description'])
               )

In [9]:
con.commit()
con.close()